In [1]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Mecab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

from Preprocess import load, text_preprocess

In [2]:
texts = pd.read_csv("./Data/csv/dummy_transcripts.csv")["text"]

In [3]:
texts = "".join(texts)

In [4]:
mecab = Mecab()

tokenized_doc = mecab.nouns(texts)
tokenized_nouns = " ".join(tokenized_doc)
tokenized_nouns

'이번 시간 인수 분해 인 수분 우리 곱셈 공식 밀접 관련 곱셈 공식 필요 다항식 뜰 곱 전개 데 필요 공식 당시 게 곳 전개 것 이인수 분 관련 우리 곱셈 공식 거 기억 거 뭐 다음 식과 다음 식 고 제곱 플러스 유 다항식 곱셈 하나 당 식 표현 수 걸 그때 이쪽 일 과정 우리 전개 표현 일쑤 분해 건 뭘 얘기 이쪽 과정 인수 분해 얘기 이 둘 차이점 뭐 전개 것 개 이상 다항식 하나 다항식 표현 거 요인 수분 것 하나 다항식 개 이상 다항식 곱 표현 게 인 수분 이때 당시 이 뭐 이우 변 방식 제곱 플러스 인수 얘기 하나 당시 글 인수 분해 그것 곱 표현 의미 우리 이름 인수 분해 거 여기 말씀 학생 질문 선생 제 걸 예 숫자 곱 표현 것 인수 분해 이것 건가요 질문 여러분 제 뭐 하나 당시 글 개 이상 다항식 곱 표현 것 인수 분해 다 말씀 다항식 게 뭐 다항식 하나 이상 다낭 식 뜰 덧셈 뺄셈 시기 우리 다낭 식도 다항식 포함 거 다낭 시계 상수 항의 것 상수 왕 숫자 존재 항의 문자 것 우리 상상 상수 항 팔 상수 항의 낭 식 낭 다항식 표현 포함 사와이 마찬가지 우리 다항식 수 팔 사 이 곱 표현 것 우리 인 수분 얘기 것 얘기 겁니다 여러분 소인수 분해 얘기 거 분해 건 뭐 예 곱 로 도 표현 수 곱 곱 로 도 표현 수 이걸 우리 의 세제곱 표현 소인수 분해 건 뭐 인수 분해 누구 인 수분 얘기 소수 인수 분해 얘기 여기 곱 이 소수 소수 앤 이사 소수 고 곱 호 인수 구매 겁니다 파리 것 수 소수 곱 표현 수 때문 소수 이용 인수 분해 것 우리 소인 수분 거 여기 인 수분 뭔지 감 겁니다 우리 다음 것 인수 분해 방법 뭐 거 지금 얘기 번 인수 분해 때 우리 게 뭐 공통 인수 겁니다 번 제목 공통 인수 거 자요 예 하나 게 요예 제곱 플러스 당시 글 수분 나 문제 생각 여기 각각 낭 이 남 들 공통 술 거 예 제곱 반항 곱 수 곱 수 안 공통 인수 걸 수 공통 인수 다음 괄호 프라스 겁니다 요거 전개 제곱 플러스 개 공통 인수 수 거 얘 요건 얘

In [5]:
n_gram_range = (2,3)
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개 수 :',len(candidates))
print('trigram 다 섯 개 만 출 력 :',candidates[:5])

trigram 개 수 : 1494
trigram 다 섯 개 만 출 력 : ['가능 그거' '가능 그거 여기' '가정 크로스' '가정 크로스 로고' '가지 가지']


In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [7]:
doc_embedding = model.encode([texts])
candidates_embeddings = model.encode(candidates)

In [8]:
top_n = 5 # 상위 5개 키워드
distances = cosine_similarity(doc_embedding, candidates_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['곱셈 교환 법칙', '밀접 관련 곱셈', '인수 결과 곱셈', '곱셈 공식 방법', '인수 시행착오 뭔가요']


## 다양한 키워드 얻기
- Max sum similarity
- Maximal Marginal Relevance

### 1. Max Sum Similarity
- 데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의 된다.
- 후보간의 유사성을 최소화 하면서 문서와의 후보 유사성을 극대화하자

In [9]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n 단어 pick
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx] 
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candicate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candicate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candicate]


In [10]:
max_sum_sim(doc_embedding, candidates_embeddings, candidates, top_n=5, nr_candidates=10)

['기본 인수 공식', '인수 인수 연습', '밀접 관련 곱셈', '인수 결과 곱셈', '인수 시행착오 뭔가요']

In [11]:
# 높은 nr_candidates는 상대적으로 다양한 키워드를 만든다
max_sum_sim(doc_embedding, candidates_embeddings, candidates, top_n=5, nr_candidates=30)

['생각 이해 연습', '일과 시도', '선생 숫자 표현', '포함 사와이 마찬가지', '밀접 관련 곱셈']

---
## CTM

In [12]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

In [13]:
loader = load.DataLoad(
    tran_path = "./Data/csv/sample_transcripts.csv",
    meta_path="./Data/csv/sample_videos.csv"
)

In [14]:
texts_dict_popular, texts_dict_unpopular = loader.get_data_set()

In [15]:
preprocessor = text_preprocess.Preprocess(stopwords=load.DataLoad.load_stopwords())

In [16]:
texts_dict = preprocessor.get_cleaned_dict(texts_dict_popular)

[Kss]: Oh! You have konlpy.tag.Mecab in your environment. Kss will take this as a backend! :D



  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/97 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

...

  0%|          | 0/820 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/151 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

In [21]:
texts = [doc for doc in texts_dict.values()]

In [28]:
texts = list(itertools.chain(*texts))

In [49]:
stopwords = load.DataLoad.load_stopwords()

In [57]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.nouns(sent)
        result = [word for word in word_tokens if len(word) > 1]
        result = [word for word in result if word not in stopwords]
        return result
    
custom_tokenizer = CustomTokenizer(Mecab())

In [58]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [63]:
from pprint import pprint
pprint(vocab.tolist())

['가게',
 '가격',
 '가결',
 '가공',
 '가나',
 '가늠',
 '가능',
 '가닥',
 '가도',
 '가락',
 '가래',
 '가로',
 '가루',
 '가르마',
 '가문',
 '가버',
 '가보세요',
 '가분',
 '가사',
 '가시',
 '가식',
 '가십',
 '가영',
 '가옥',
 '가운',
 '가운데',
 '가을',
 '가음',
 '가의',
 '가이',
 '가인',
 '가입',
 '가정',
 '가질',
 '가짜',
 '가치',
 '가평',
 '각계',
 '각도',
 '각항',
 '각형',
 '간격',
 '간식',
 '간암',
 '간장',
 '갈라',
 '감각',
 '감사',
 '감수',
 '감안',
 '감옥',
 '감자',
 '감정',
 '갑자',
 '강가',
 '강령',
 '강산',
 '강스',
 '강의',
 '강자',
 '강조',
 '강좌',
 '개개',
 '개관식',
 '개념',
 '개띠',
 '개로',
 '개면',
 '개봉',
 '개선',
 '개설',
 '개소',
 '개소리',
 '개수',
 '개야',
 '개업',
 '개요',
 '개인',
 '개인전',
 '개입',
 '개적',
 '개점',
 '개정',
 '개중',
 '개체',
 '객관식',
 '객실',
 '객체',
 '거듭제곱',
 '거란',
 '거랑',
 '거래',
 '거리',
 '거부',
 '거시',
 '거야',
 '거제',
 '거주',
 '거즈',
 '거지',
 '거함',
 '걱정',
 '건데요',
 '건마',
 '걸까',
 '걸요',
 '걸음',
 '검날',
 '검사',
 '검산',
 '검색',
 '검정',
 '검증',
 '검지',
 '겁니까',
 '겁니다',
 '게살',
 '게이',
 '게이고',
 '게임',
 '겨울',
 '결계',
 '결과',
 '결과물',
 '결구',
 '결단',
 '결론',
 '결실',
 '결정',
 '결함',
 '경계',
 '경기',
 '경련',
 '경험',
 '경희대',
 '계곡',
 '계단',
 '계산',
 '계선',
 '계수',
 '계약

In [53]:
train_bow_embeddings = vectorizer.fit_transform(texts)


In [62]:
print(train_bow_embeddings.shape)


(4453, 3000)


In [64]:
vocab = vectorizer.get_feature_names_out()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

NotFittedError: Vocabulary not fitted or provided

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(texts, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")
     

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

In [38]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)
     

In [46]:

ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=20, num_epochs=20)
ctm.fit(training_dataset)
     

Epoch: [20/20]	 Seen Samples: [89060/89060]	Train Loss: 63.162948707770745	Time: 0:00:47.143565: : 20it [15:47, 47.35s/it]


In [48]:

ctm.get_topics(5)


defaultdict(list,
            {0: ['도록', '겁니다', '예요', '당연히', '세제곱'],
             1: ['다음', '제곱', '곱하', '으로', '이렇게'],
             2: ['에요', '편의', '가로', '잖아요', '그쵸'],
             3: ['다음', '해서', '우리', '공식', '인수'],
             4: ['분해', '그래서', '일단', '어떤', '으로'],
             5: ['제곱', '는데', '이렇게', '라는', '라고'],
             6: ['꺼려', '공동', '으러', '엄청', '어떻'],
             7: ['니까', '이렇게', '여기', '플러스', '라고'],
             8: ['할머니', '파형', '약품', '자비', '하프'],
             9: ['할머니', '기술', '자비', '파형', '이쪽'],
             10: ['제곱', '으로', '플러스', '다음', '니까'],
             11: ['잖아', '얘기', '그냥', '근데', '전체'],
             12: ['기능', '할머니', '꼬인', '하프', '기술'],
             13: ['마음', '일생', '조리', '그러나', '색깔'],
             14: ['상승', '할머니', '기능', '자비', '꺼릴'],
             15: ['제곱', '완전', '그냥', '으니까', '그럼'],
             16: ['제곱', '플러스', '다음', '니까', '곱하'],
             17: ['에서', '인수', '공식', '수분', '여러분'],
             18: ['여러분', '에서', '공식', '문제', '과정'],
             19: ['기능', '하프', '참고', '꼬인', 

---
## BERTopic

In [65]:

from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic

In [66]:
texts

['안녕하세요',
 '음악 오늘은 분 술에 선수로 따 초만에 닦은 다음 법 음악 자아 이거 폼 여는 클랑 약  채워 채워줘요',
 '어떻게 이렇게 빨리할 수 있느냐 우리가 분수를 소수로 바꾸는 이유는 뭐예요',
 '우리가 쉽진 들었으니까 순위 크기를 가늠할 수가 있죠',
 '근데 보통 이걸로도 가능해요',
 '아 반보다는 크구나',
 '근데 이제 정확히는 모르죠',
 '팜 범행이라고 하면 셋 중에 툴 정도 되겠구나',
 '뭐 이렇게까지 날 수 있는데 이제 정확하게 나타내 라 소수로 일반적으로는 어떻게 풀어 저게 씻고 나누기  이니까 우로 나눈 미샤 하나 붙이고 뭐 이렇게 가지고 빼고 음악 이렇게 해서 복잡하게 이제 풀어요',
 '풀다 보면 은 이게 이제 반복이 되거든요',
 '그럼 아 이건 반복 때문에 이제 그때 가서 반복되는 것을 쓰는데 이렇게 하면 안 돼요',
 '칼로 합니다',
 '이 문제를 보면은 소수라는 거에 의미를 알아야 돼요',
 '우리가 쓰면 수 진수 일 십백천 많이 몇 개 있냐',
 '이 말이요',
 '그런데 소수 니까 보다 작을 때는 다시 말해서 일 분의  천분의 일동으로 나누어 팩을 하는 첫눈 오다는 것들이 몇 개냐 로 나타내면 쉽죠',
 '그래도 우리가 이런 거 아쉬웠죠',
 '분에  이것은 곧바로 분위를 곱해 쓰니까 여기 다도를 곱하면 은 신이 그래가지고 이렇게 바로 그랬죠',
 '그래서 아내가 붐 무가 일 십백천만 원 나타내야 되는데 은 여기 하에  로 있어서 절대  백철만 으로 나타낼 수 없어요',
 '수영 많은 대 첨부 다 쉬운 수는 무엇일까 음악 태풍 어려운 사람은 바로 말  왜 잡으니까 그래서 이것은 그대로 위 알의 을 곱하면 구식 분의  윤 월 부품에는 인 을 빼면 이제는 것은 왜 주었어요',
 '바로   다시 말해서  을 으로 나누는데 하나를 남겨 놓고 나는 하는 거 보고 싶지를 으로 남았어',
 '그럼  이 줘',
 '그런데 한계를 빼빼로 않았으니까 이 이거 개가 그대로 남아있네요',
 '이 말이죠',
 '그래서 이게 원래

In [68]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.nouns(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result
    
custom_tokenizer = CustomTokenizer(Mecab())

In [69]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [70]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=50,
                 top_n_words=10,
                 calculate_probabilities=True)

In [71]:
topics, probs = model.fit_transform(texts)


In [72]:
model.visualize_topics()


In [73]:
model.visualize_distribution(probs[0])


In [74]:
for i in range(0, 50):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('인수', 0.03092274552523218), ('제곱', 0.028094043900919425), ('분해', 0.026984671247785644), ('플러스', 0.02356075948419046), ('하나', 0.02002584907306349), ('공식', 0.01894297966310428), ('이용', 0.018942818935379233), ('분자', 0.01787327825703319), ('수분', 0.01704720746029579), ('복소수', 0.0165941444891013)]
1 번째 토픽 : [('나머지', 0.01937214029713615), ('이거', 0.019266592809407627), ('다음', 0.0166420521871706), ('선생', 0.014723377534046716), ('여기', 0.014134551102670213), ('제곱', 0.014034810688259952), ('플러스', 0.013566203591887543), ('생각', 0.012355872174402644), ('이게', 0.011773599451036077), ('얘기', 0.011682082200124845)]
2 번째 토픽 : [('예정', 0.09616628283655299), ('박수', 0.06818710218189587), ('에스', 0.054593371522327426), ('어미', 0.054593371522327426), ('파면', 0.054593371522327426), ('회답', 0.054593371522327426), ('식이', 0.054593371522327426), ('과일', 0.054593371522327426), ('환영', 0.054593371522327426), ('타령', 0.054593371522327426)]
3 번째 토픽 : [('수학', 0.059507154354550146), ('중학교', 0.04769119441083245), ('공부'